In [ ]:
ENV["JULIA_PKG_SERVER"] = "https://jp.pkg.julialang.org"

using DifferentialEquations
using Interpolations
using Plots

using LinearAlgebra

using Pkg
using CSV
using DataFrames

using Random

In [ ]:
T0 = 24
GL_tilde(t) = 1.30604 + 0.227902 * sin((2*π/T0)*(t - 3.12764))
TOR_tilde(t) = 1.01103 + 0.0244438 * sin((2*π/T0)*(t - 14.4926))

In [ ]:
Growth_rate_data = CSV.read("GrowthRate.csv", DataFrame)
max_exp = maximum(Growth_rate_data.value)
min_exp = minimum(Growth_rate_data.value)

# Define simple model

In [ ]:
function Reduced(du,u,p,t)
  # p[1]: Glucose
#   p[2]: TOR total
#   p[3]: X
#   u[1]: TOR-
  # u[2]: R
    
    n=2
    TOR = p[2](t) - u[1]
    du[1] = -p[4] * p[1](t) * u[1] + p[5] * TOR * (p[3](t-p[9]) + p[10]) - p[6] * u[1]
    du[2] = p[7] + TOR^n / (p[8]^n + TOR^n)
end

In [ ]:
tspan = (0,168)
T0 = 24
X = t -> 0.5 * cos(2 * (pi/T0)*t) + 0.5
k_act_init = 1
k_inact_init = 1
d_TORm_init = 1
g_init = 0.35 
K_init = 1
tau_init = 12
B_init = 2

p = (GL_tilde, TOR_tilde, X, k_act_init, k_inact_init, d_TORm_init, g_init, K_init, tau_init, B_init)

u_0 = [0, 200]                       # initial state vector
prob = ODEProblem(Reduced,u_0,tspan,p)
fit = solve(prob,Vern9(lazy=false); saveat =0.1)

# Prameter generation

In [ ]:
using Random
using Turing 
Random.seed!(14);

L = 2500
k_act = 10 .^ rand(Uniform(-1, 1), L)
k_inact = 10 .^ rand(Uniform(-1, 1), L)
d_TORm = 10 .^ rand(Uniform(-1,1),L)
g = 10 .^ rand(Uniform(-1,1), L)
K = 10 .^ rand(Uniform(-1,1),L)

S = 500
params = zeros(5,S)

for i in 1:S
    params[1,i] = k_act[rand(1:end)]
    params[2,i] = k_inact[rand(1:end)]
    params[3,i] = d_TORm[rand(1:end)]
    params[4,i] = g[rand(1:end)]
    params[5,i] = K[rand(1:end)]
end

In [ ]:
tau_ = 5 * ones(S)
Amp_ = 0.5 * ones(S)
params_space = vcat(params, tau_', Amp_')

prob_func = let p=p
    (prob,i,repeat) -> begin
        remake(prob, p=(GL_tilde, TOR_tilde, X, params_space[:,i]...))
    end
end

@time begin
ensemble_prob = EnsembleProblem(prob,prob_func=prob_func)
sim = solve(ensemble_prob,Tsit5(), trajectories= S)
end

# Calculate the error between simulation and experiment

In [ ]:
num = 50 
time_points = Growth_rate_data.time
time_diff = time_points[2] - time_points[1]
matrix = zeros(S, length(time_points))

tau = range(0, stop = 24, length = num)
Amp = range(1/num, stop = 1, length = num)
error = ones(num, num)
error_temp = ones(S)

@time begin
for j in 1:num
    for k in 1:num
        prob_func = let p=p
            (prob,i,repeat) -> begin
                remake(prob, p=(GL_tilde, TOR_tilde, X, params[:,i]...,tau[j], 1/Amp[k] - 1))
            end
        end

        @time begin
        ensemble_prob = EnsembleProblem(prob,prob_func=prob_func)
        sim = solve(ensemble_prob,BS3(), trajectories= S)
        end

        for i in 1:S
            u0 = sim[i](time_points[1] + 120 - time_diff, idxs = 2)
            u1 = sim[i](time_points[1] + 120, idxs = 2)
            matrix[i,1] = (u1-u0)/time_diff
            for t in 2:length(time_points)
                u0 = sim[i](time_points[t-1] + 120, idxs = 2)
                u1 = sim[i](time_points[t] + 120, idxs = 2)
                dt = time_points[t] - time_points[t-1]
                matrix[i,t] = (u1-u0)/dt
            end
        end

        for i in 1:S
            R_sim = matrix[i,:]
            min_sim = minimum(R_sim)
            max_sim = maximum(R_sim)
            R_scale = ( (max_exp - min_exp) / (max_sim - min_sim) ) .* (R_sim .- min_sim) .+ min_exp
            error_temp[i] = norm(R_scale - Growth_rate_data.value)
        end
        
        error[j,k] = minimum(error_temp) * (1/sqrt(length(time_points)))
        
        print(join([j,",",k]))
    end
end
end